## Algoritmos de clustering
   
    # Fuzzy C-Means - FCM
    # Kernel Fuzz C-Means - KFCM-K
    # Variable-Wise Kernel Fuzz C-Means - VKFCM-K
   
#### No dataset Image_Segmentation UCI
##### http://archive.ics.uci.edu/ml/datasets/image+segmentation

In [2]:
import numpy as np
import pandas as pd

# Algoritmos de cluster
from fcm import FCM
from kfcm import KFCM
from vkfcm import VKFCM

# Algoritmo base para comparação K-Means
from sklearn.cluster import KMeans

# metricas
from sklearn.metrics import adjusted_rand_score

### Read data

In [3]:
df = pd.read_csv("./data/segmentation.data")
df.head(5)

,CLASSE,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
0,BRICKFACE,140.0,125.0,9,0.0,0.0,0.277778,0.062963,0.666667,0.311111,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818
1,BRICKFACE,188.0,133.0,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
2,BRICKFACE,105.0,139.0,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
3,BRICKFACE,34.0,137.0,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
4,BRICKFACE,39.0,111.0,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773


### Analise da base

In [4]:
df.describe()

,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
count,210.000000,210.000000,210.0,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,124.647619,122.757143,9.0,0.008466,0.006349,1.925132,5.719529,2.604233,11.638377,37.091005,32.967725,44.011112,34.294180,-12.369841,20.760317,-8.390476,44.888360,0.423230,-1.340397
std,74.104024,58.139686,0.0,0.029549,0.030077,3.158211,43.495942,4.798268,97.390023,38.677168,35.540563,43.804447,37.057003,11.559599,18.761842,11.003746,43.235182,0.227333,1.559405
min,1.000000,11.000000,9.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-48.222220,-9.666667,-30.555555,0.000000,0.000000,-2.530950
25%,60.500000,81.500000,9.0,0.000000,0.000000,0.666667,0.400921,0.777779,0.410816,6.453704,7.000000,8.277778,3.805555,-18.111110,4.111111,-15.750000,10.527778,0.275722,-2.187112
50%,123.500000,121.500000,9.0,0.000000,0.000000,1.222222,0.828695,1.388889,0.913176,21.314816,18.611112,26.833334,20.000000,-10.333333,19.555556,-9.888889,28.388890,0.365455,-2.044163
75%,189.750000,174.500000,9.0,0.000000,0.000000,1.888890,1.676634,2.597221,1.980485,52.629629,46.750000,64.194447,46.472223,-4.666666,34.333332,-3.722222,64.194447,0.539738,-1.429800
max,252.000000,250.000000,9.0,0.111111,0.222222,25.500000,572.996400,44.722225,1386.329200,143.444440,136.888890,150.888890,142.555560,5.777778,78.777780,21.888890,150.888890,1.000000,2.864931


In [5]:
# Classe balanceada
df['CLASSE'].value_counts()

GRASS        30
FOLIAGE      30
SKY          30
PATH         30
CEMENT       30
WINDOW       30
BRICKFACE    30
Name: CLASSE, dtype: int64

### A base será analisada por 2 Views
     Shape view - 9 primeiros atributos
     RGB view - 10 ultimso atributos

In [6]:
df.shape

(210, 20)

In [7]:
shape_view = df.iloc[:, 1:-10]
rgb_view =  df.iloc[:, -10:]
target = df.iloc[:, 0]

In [8]:
shape_view.head(1)

,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD
0,140.0,125.0,9,0.0,0.0,0.277778,0.062963,0.666667,0.311111


In [9]:
rgb_view.head(1)

,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
0,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818


## Cluster parameters

In [10]:
num_clusters = 7
m = 1.6
max_iter = 150 
e =10^(-10)

### KMeans

In [11]:
kmeans_shape = KMeans(n_clusters=num_clusters).fit(shape_view)
kmeans_rgb = KMeans(n_clusters=num_clusters).fit(rgb_view)

In [12]:
y_pred_shape = kmeans_shape.predict(shape_view)
y_pred_rgb = kmeans_rgb.predict(rgb_view)
print("Shape view : ", adjusted_rand_score(target, y_pred_shape))
print("RGB view : ", adjusted_rand_score(target, y_pred_rgb))

Shape view :  0.20218077097259907
RGB view :  0.37866089740300374


### FCM

In [13]:
fcm_shape = FCM(shape_view.values, 7, m=m)

for i in range(max_iter):
    fcm_shape.update_centroids()
    fcm_shape.update_membership()
    
fcm_rgb = FCM(rgb_view.values, 7, m=m)

for i in range(max_iter):
    fcm_rgb.update_centroids()
    fcm_rgb.update_membership()


In [14]:
y_pred_shape = fcm_shape.predict(shape_view)
print("Shape view : ", adjusted_rand_score(target, y_pred_shape))

y_pred_rgb = fcm_rgb.predict(rgb_view)
print("Shape view : ", adjusted_rand_score(target, y_pred_rgb))

Shape view :  0.21588964471525618
Shape view :  0.37866089740300374


### VKFCM

In [15]:
vkfcm_shape = VKFCM(shape_view.values, 7, m=1.6)

for i in range(150):
    vkfcm_shape.update_centroids()
    vkfcm_shape.update_weights()
    vkfcm_shape.update_membership()
    
vkfcm_rgb = VKFCM(rgb_view.values, 7, m=1.6)

for i in range(150):
    vkfcm_rgb.update_centroids()
    vkfcm_rgb.update_weights()
    vkfcm_rgb.update_membership()

In [17]:
y_pred_shape = vkfcm_shape.predict(shape_view.values)
print("Shape view : ", adjusted_rand_score(target, y_pred_shape))

y_pred_rgb = vkfcm_rgb.predict(rgb_view.values)
print("Shape view : ", adjusted_rand_score(target, y_pred_rgb))

Shape view :  0.0
Shape view :  0.4103782755558042
